In [ ]:
import os
os.environ['QT_QPA_PLATFORM'] = 'xcb'
!wget https://people.csail.mit.edu/billf/project%20pages/sresCode/Markov%20Random%20Fields%20for%20Super-Resolution_files/iccv.maddie.small.jpg

In [ ]:
!wget -L https://raw.githubusercontent.com/Saafke/EDSR_Tensorflow/master/models/EDSR_x2.pb


In [ ]:
import cv2
import skimage
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread('iccv.maddie.small.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

print("Original Image Shape:", img.shape)
plt.imshow(img)

In [ ]:
def interpolation(img, mode_type, factor=None):
    if mode_type == "Cubic Spline":
        # only function that uses the skiimage instead of the cv2
        img = skimage.transform.resize(img, (img.shape[0] * factor, img.shape[1] * factor), order=3, anti_aliasing=Fasle)
        return img

    elif mode_type == "EDSRx4":
        print(f"This model only supports scale factor = 2, so all the images will have the size multiplied by 2.")

        sr = cv2.dnn_superres.DnnSuperResImpl_create()

        path = "EDSR_x2.pb"
        sr.readModel(path)
        sr.setModel("edsr", 4)

        result = sr.upsample(img)
        return result

    elif mode_type == "Linear":
        mode = cv2.INTER_LINEAR
    elif mode_type == "Cubic":
        mode = cv2.INTER_CUBIC
    elif mode_type == "Nearest":
        mode = cv2.INTER_NEAREST
    elif mode_type == "Lanczos4":
        mode = cv2.INTER_LANCZOS4

    img = cv2.resize(img, (0,0), fx=factor, fy=factor, interpolation=mode)
    return img

modes = ["Nearest", "Linear", "Cubic", "Cubic spline", "Lanczos4", "EDSRx4"]
factors = [
    2
]
image_dict = {}

for mode in modes:
    for factor in factors:
        if mode == "EDSRx4":
            image_dict[mode] = interpolation(img, mode)
        else:
            image_dict[f"{mode} x{factor}"] = interpolation(img, mode, factor)

titles = list(image_dict.keys())
data = list(image_dict.values())

fig, axes = plt.subplots(6, 2, figsize=(12,30))
for i in range(6):
    for j in range(2):
        index = i*2 + j
        axes[i, j].imshow(data[index])
        axes[i, j].set_title(titles[index])
        axes[i, j].set_xlabel("Weight")
        axes[i, j].set_ylabel("Height")

fig.suptitle("Interpolation across different frames", fontsize=20)
plt.tight_layout()
plt.show()
    